# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [ ]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
from datetime import datetime
import numpy as np

In [ ]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'
# url = 'COVID-19.csv'
df = pd.read_csv(url)


In [ ]:
# Ensuring the Date is just date, and not datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

In [ ]:
# examining columns and rows
df.shape

In [ ]:
# checking type for each column
df.info()

In [ ]:
# examining the names of the columns
df.columns

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [ ]:
df['X'].unique()

In [ ]:
df['Y'].unique()

In [ ]:
df['CommunityTransmission'].unique()

In [ ]:
df['CloseContact'].unique()

In [ ]:
df['TravelAbroad'].unique()

### Dropping Columns (X and Y)

| Column | Reason |
|---|---|
| df['X'] | no unique values |
| df['Y'] | no unique values |
| df['StatisticsProfileDate'] | Very similar to df['Date']|
| df['CovidCasesConfirmed'] | Very similar to df['Total Confirmed Covid Cases'] |
| df['FID'] | very similar to Index in pandas |
| df['CommunityTransmission'] | no unique values |
| df['CloseContact'] | no unique values |
| df['TravelAbroad'] | no unique values |


In [ ]:
# Dropping irrelevant columns
df = df.drop(columns = ['X','Y','CovidCasesConfirmed','StatisticsProfileDate', 'FID', 
                        'CommunityTransmission', 'CloseContact','TravelAbroad'])

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [ ]:
df['Date'].unique()

## Melting 

In [ ]:
# Melting
id_vars = ["Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="Column", value_name="Cases")

df.head(10)


# Getting the final dataset

In [ ]:
df.head()

In [ ]:
df['Age_Range'] = df['Column']
df.tail()

In [ ]:
df['Age_Range'] = df['Age_Range'].str.replace('to','-')

In [ ]:
df['Age_Range']= df['Age_Range'].str.replace('up','+')

In [ ]:
df['Age_Range']

In [ ]:

df['Age_Range']= df['Age_Range'].replace("TotalConfirmedCovidCases", np.nan)
df['Age_Range']= df['Age_Range'].replace("ConfirmedCovidCases",np.nan)
df['Age_Range']= df['Age_Range'].replace("TotalCovidDeaths", np.nan)
df['Age_Range']= df['Age_Range'].replace("ConfirmedCovidDeaths",np.nan)
df['Age_Range']= df['Age_Range'].replace("HospitalisedCovidCases", np.nan)
df['Age_Range']= df['Age_Range'].replace("RequiringICUCovidCases", np.nan)
df['Age_Range']= df['Age_Range'].replace("HealthcareWorkersCovidCases", np.nan)
df['Age_Range']= df['Age_Range'].replace("ClustersNotified", np.nan)
df['Age_Range']= df['Age_Range'].str.replace("Aged", '')
df['Age_Range']= df['Age_Range'].str.replace("Hospitalised5", '1-4')
df['Age_Range']= df['Age_Range'].str.replace("1-4-14",'5-14')
df['Age_Range']= df['Age_Range'].str.replace("1-45-64",'55-64')
df['Age_Range']= df['Age_Range'].str.replace("Hospitalised", '')
df['Age_Range']= df['Age_Range'].replace("Median_Age", np.nan)
df['Age_Range']= df['Age_Range'].replace("Male", np.nan)
df['Age_Range']= df['Age_Range'].replace("Female", np.nan)
df['Age_Range']= df['Age_Range'].replace("Female", np.nan)
df['Age_Range']= df['Age_Range'].replace("Unknown",np.nan)

In [ ]:
df["Age_Range"].unique()

In [ ]:
df["Column"].unique()

In [ ]:
df = df [[
    "Date",
    "Column",
    "Age_Range",
    "Cases"
]]

df.head()

In [ ]:
#        'HospitalisedAged5', 'HospitalisedAged5to14',
#        'HospitalisedAged15to24', 'HospitalisedAged25to34',
#        'HospitalisedAged35to44', 'HospitalisedAged45to54',
#        'HospitalisedAged55to64', 
#        'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44',
#        'Aged45to54', 'Aged55to64',


# checking that 'HospitalisedAged5' with have 5
df[df["Column"] =='HospitalisedAged55to64'].head()

In [ ]:
# checking that 'HospitalisedAged85up' with have 85
df[df["Column"] =='HospitalisedAged85up'].head()

### Dropping rows with NaN in cases

In [ ]:
df.head(20)

In [ ]:
df = df[~df['Cases'].isna()]
df["Cases"] = df["Cases"].astype(int)
# reseting the dataframe index
df = df.reset_index(drop = True)

In [ ]:
df.head()

In [ ]:
df.shape

### Dealing with ['Column'] column

In [ ]:
# Testing the regular expression
line = 'HospitalisedAged45to54'
match = re.findall('([A-Z][a-z]+)', line)
new_line = match[0] + " " + match[1] 
print(new_line)

In [ ]:
column_list = []
for i in  df["Column"]:
    match = re.findall('([A-Z][a-z]+)', i)
    if len(match) == 4:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2] + ' ' + match[3]
    elif len(match) == 3:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2]
    elif len(match) == 2:
        new_line = match[0] + ' ' + match[1] 
    else:
        new_line = match[0]

    column_list.append(new_line)

# examining the list
print(column_list)  

In [ ]:
column_list = pd.Series(column_list)
column_list = column_list.reset_index()

In [ ]:
df = pd.merge(df, column_list, left_index = True, right_index = True)

In [ ]:
df = df.rename(columns={'Column': "old_column", 0 : "Column"})

In [ ]:
df = df[[
    "Date",
    "old_column",
    "Column",
    "Age_Range",
    "Cases"
]]

In [ ]:
df.head()

### Daily Cases

In [ ]:
hospitalised_df = df[df['Column'].str.contains("Hospitalised Aged")]
age_range_65up = hospitalised_df[hospitalised_df['Age_Range'] == '65-74']
age_range_65up.tail(10)

In [ ]:
age_list = df['Age_Range'].unique()

# removing NaN from the dataset
age_list = age_list[1:]
print(age_list)

In [ ]:
aged_df = df[df['Column'].str.startswith("Aged")]

# dropping all NaN in hospitalised_df 
hospitalised_df = hospitalised_df[~hospitalised_df['Age_Range'].isna()]

# dropping all NaN in aged_df
aged_df = aged_df[~aged_df['Age_Range'].isna()]

# dropping all NaN in df1
new_df = df[~df['Column'].isna()]

In [ ]:
aged_df.head()

### Columns that are Not Accumulated
'Confirmed Covid Cases', 'Total Confirmed Covid Cases','Confirmed Covid Deaths', 'Total Covid Deaths',

In [ ]:
df['Column'].unique()

In [ ]:
column_list = df['Column'].unique()
column_list

In [ ]:
# Columns that have accumulated cases and no daily cases count 
accumulated = ['Male','Hospitalised Covid Cases', 'Hospitalised Aged', 
               'Healthcare Workers Covid Cases', 'Aged', 'Unknown', 'Female',
               'Median Age', 'Requiring Covid Cases', 'Clusters Notified']

In [ ]:
df_list = []

for i in column_list:
    if "Hospitalised Aged" == i: 
        for age in age_list:
            df = hospitalised_df[hospitalised_df['Age_Range'].str.contains(age)]
            df = pd.DataFrame(df)
            df['Daily Cases'] = df['Cases'].diff(1)
            df['Diff'] = df['Daily Cases'].diff(1)
#             df = df.dropna()
#             df["Daily Cases"] = df["Daily Cases"].astype(int)
#             df["Diff"] = df["Diff"].astype(int)
            df_list.append(df)
        
    elif "Aged" == i: 
        for age in age_list:
            df = aged_df[aged_df['Age_Range'].str.contains(age)]
            df = pd.DataFrame(df)
            df['Daily Cases'] = df['Cases'].diff(1)
            df['Diff'] = df['Daily Cases'].diff(1)
#             df = df.dropna()
#             df["Daily Cases"] = df["Daily Cases"].astype(int)
#             df["Diff"] = df["Diff"].astype(int)
            df_list.append(df)
    
    elif  i in accumulated:
        df = new_df[new_df['Column'].str.contains(i)]
        df = pd.DataFrame(df)
        df['Daily Cases'] = df['Cases'].diff(1)
        df['Diff'] = df['Daily Cases'].diff(1)
#         df = df.dropna()
#         df["Daily Cases"] = df["Daily Cases"].astype(int)
#         df["Diff"] = df["Diff"].astype(int)
        df_list.append(df)
    else:
        df = df[df['Column'].str.contains(i)]
        df = pd.DataFrame(df)
        df['Daily Cases'] = 'NaN'
        df['Diff'] = df['Cases'].diff(1)
#         df = df.dropna()
#         df["Daily Cases"] = df["Daily Cases"].astype(int)
#         df["Diff"] = df["Diff"].astype(int)
        df_list.append(df) 

In [ ]:
df =  pd.concat(df_list)

In [ ]:
# sorting data by date
df = df.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df = df.reset_index(drop=True)

In [ ]:
df.head(50)

In [ ]:
df.tail(50)

### Saving to a new csv file

In [ ]:
df.to_csv(r"tidy_data_df.csv", index =  False)